<a href="https://colab.research.google.com/github/g-miroslav/freecodecamp/blob/main/Machine_Learning_with_Python/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# 1. convert categorical values into numeric
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'no': 0, 'yes': 1})
dataset['region'] = dataset['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})

dataset.tail()

In [ ]:
# 2. Split data into training (80%) and testing (20%) datasets (dataframes)
train_dataset = dataset.sample(frac = 0.8, random_state = 0) # random_state makes sure the same data is in both datasets when run multiple times
test_dataset = dataset.drop(train_dataset.index) # Remove the training dataset from from the original dataset to get the test dataset

In [ ]:
# 3. Remove the column 'expenses' from the training and testing datasets and place them into new datasets for labels (outputs)
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

In [ ]:
# 4. Create normalization layer
normalizer = tf.keras.layers.Normalization()

In [ ]:
# 5. Fit the state of the preprocessing layer to the data - computes the mean and variance of values
normalizer.adapt(np.array(train_dataset))
print(normalizer.mean.numpy())

In [ ]:
# 6. Build Keras Sequential model
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(1)
])

In [ ]:
# 7. Configure the training procedure, particularly the loss and the optimizer
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    loss='mean_absolute_error',
    metrics=['mae', 'mse']
)

In [ ]:
# 8. Use Keras Model.fit to execute the training for 100 epochs
history = model.fit(
    train_dataset,
    train_labels,
    epochs = 100,
    verbose = 0 # suppress logging
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
